In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from langdetect import detect

pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('offers_v3.csv')
df = df.drop(['id'], axis=1)

In [3]:
len(df)

4445

In [4]:
geo = pd.read_excel('geotag.xlsx')

In [5]:
# Remove Chinese offers
i = 0
while i < len(df)-1:
    x = df.iloc[i, :]
    if len(re.findall(r'[\u4e00-\u9fff]+', x['name'])) > 0:
        df = df.drop([i])
    i = i + 1
df = df.reset_index(drop=True)

In [6]:
# Detect Languages
df['lang'] = ''

i = 0
while i <len(df)-1:
    try:
        x = df.iloc[i, :]['desc']
        df['lang'][i] = detect(x)
    except:
        x = df.iloc[i, :]['name']
        df['lang'][i] = detect(x)
    i = i + 1
print('Done')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Done


In [ ]:
#df.to_excel('offers_extract.xlsx')

In [7]:
df['details'] = df['details'].apply(lambda x : str(x))
df = df.reset_index(drop=True)

contract_type = []
ct1 = []
ct2 = []
ct3 = []
x = 0
while x <= df.shape[0]-1:
    target = df['details'][x]
    
    c = re.findall('cdi|cdd|durée déterminée|durée indéterminée|regular full time', target.lower())
    if not c:
        ct1.append(c)
        #print(c)
    else:
        if c[0].upper() == 'DURÉE INDÉTERMINÉE':
            ct1.append('CDI')
            #print(c[0].upper())
        elif c[0].upper() == 'DURÉE DÉTERMINÉE':
            ct1.append('CDD')
            #print(c[0].upper())
        elif c[0].upper() == 'REGULAR FULL TIME':
            ct1.append('CDI')
            #print(c[0].upper())
        else:
            ct1.append(c[0].upper())
            #print(c[0].upper())

    targ = df['desc'][x]
    
    try:  
        r = re.findall('cdi|cdd|durée déterminée|durée indéterminée', targ.lower())
        if not r:
            ct2.append(r)
            #print(r)
        else:
            if r[0].upper() == 'DURÉE INDÉTERMINÉE':
                ct2.append('CDI')
                #print(r[0].upper())
            elif r[0].upper() == 'DURÉE DÉTERMINÉE':
                ct2.append('CDD')
                #print(r[0].upper())
            else:
                ct2.append(r[0].upper())
                #print(r[0].upper())
    except:
        ct2.append('')
            
    search_field = df['name'][x]
    
    res = re.findall('cdi|cdd|durée déterminée|durée indéterminée', search_field.lower())
    if not res:
        ct3.append(res)
        #print(r)
    else:
        if res[0].upper() == 'DURÉE INDÉTERMINÉE':
            ct3.append('CDI')
            #print(r[0].upper())
        elif res[0].upper() == 'DURÉE DÉTERMINÉE':
            ct3.append('CDD')
            #print(r[0].upper())
        else:
            ct3.append(res[0].upper())
            #print(r[0].upper())
    x = x + 1

x = 0
while x <= df.shape[0]-1:
    if not ct1[x]:
        if not ct2[x]:
            if not ct3[x]:
                contract_type.append('Unspecified')
            else:
                contract_type.append(ct3[x])
        else:
            contract_type.append(ct2[x])
    else:
        contract_type.append(ct1[x])
    x = x + 1
    
df['contract_type'] = contract_type

In [8]:
pays = []
region = []
dpt = []
ville = []

id_algo = ' [x]'

x = 0
while x <= df.shape[0]-1:
    target = df['details'][x]
    
    v = re.findall('bengaluru|hénin beaumont|hénin-beaumont|bengaluru|mexico|huntington beach|chengdu|suzano|waltham|bologna|fecamp|fécamp|dieppe|allston|shenzhen|chevilly-larue|vannes|villeurbanne|la queue-en-brie|magny-les-hameaux|reims|london|la rochelle|cwmbran|canton|chorzów|vitry-sur-seine|barcelona|barcelone|bangkok|buenos aires|johannesbourg|midrand|atlanta|torino|prague|lisbonne|stuttgart|sydney|nur-sultan|budapest|détroit|henin beaumont|amiens|wissous|waterford|djeddah|hengelo|bayonne|saint-sébastien-sur-loire|austin|méxico|singapour|madrid|new york|ocoyoacac|zeralda|saint-chamond|karlstein|thonon-les-bains|charlotte|blackburn|rydalmere|barcelone|coblence|charleroi|francfort-sur-le-main|oslo|berlin|melbourne|toronto|münchen|cairo|bucarest|singapore|savigliano|copenhagen|swiftwater|framingham|moscow|temara|pôrto alegre|bridgewater|noida|cambridge|chorzow|brea|cleveland|bucuresti|bangalore|shanghai|lille|marseille|saint nazaire|saint-nazaire|portet sur garonne|st brieuc|wattrelos|compiègne|bouguenais|dampierre en burly|dampierre-en-burly|angers|albertville|massy|libourne|pau|saumur|saint-denis|grenoble|dhari|rungis|gardanne|lens|ollioules|bagneux|nevers|mérignac|montréal|montreal|saint-ouen|saint ouen|epinal|soliman|carquefou|bordes|rouen|pontarlier|velizy|gennevilliers|montpellier|eyguières|saint lô|tulle|le havre|lannion|niort|saint brieuc|lorient|arras|auxerre|colombelles|pont-audemer|feyzin|issy-les-moulineaux|palaiseau|cholet|limours|puilboreau|mantes la jolie|alençon|st maximin|pierrelatte|bourg en bresse|metz|laval|granville|avranches|toulon|beauvais|gignac|langueux|beaucouzé|la chapelle saint ursin|roanne|st sebastien|saint denis|abbeville|domerat|la défense|la defense|nanterre|lisieux|bagnolet|bourges|haguenau|saran|nantes|brive-la-gaillarde|menton|charleville-mézières|saint-dizier|versailles|toulouse|thionville|chauny|le caire|valenciennes|pantin|avignon|rians|aubervilliers|caen|rennes|courbevoie|annecy|la courneuve|le plessis-robinson|orléans|ivry sur seine|brest|eragny|cherbourg|paris|lyon|vélizy-villacoublay|élancourt', target.lower())
    if not v:
        ville.append('')
        dpt.append('')
        region.append('')
        pays.append('')
    else:
        if v[0].title() == 'Paris': ville.append('Paris'), dpt.append('Paris'),region.append('Île-de-France'), pays.append('France')
        elif v[0].title() == 'Cherbourg': ville.append('Cherbourg'), dpt.append('Manche'), region.append('Normandie'), pays.append('France')
        elif v[0].title() == 'Le Plessis-Robinson': ville.append('Le Plessis-Robinson'), dpt.append('Hauts-de-Seine'), region.append('Île-de-France'), pays.append('France')
        elif v[0].title() == 'Versailles': ville.append('Versailles'), dpt.append('Yvelines'), region.append('Île-de-France'), pays.append('France')
        elif v[0].title() == 'Bangalore' or v[0].title() == 'Bengaluru': ville.append('Bangalore'), dpt.append('Bangalore'), region.append('Karnataka'), pays.append('Inde')
        elif v[0].title() == 'Bourges': ville.append('Bourges'), dpt.append('Cher'), region.append('Centre-Val de Loire'), pays.append('France')
        elif v[0].title() == 'Cairo' or v[0].title() == 'Le Caire': ville.append('Le Caire'), dpt.append('Le Caire'), region.append('Le Caire'), pays.append('Égypte')
        elif v[0].title() == 'Saint-Ouen' or v[0].title() == 'Saint Ouen' or v[0].title() == 'Saint- Ouen': ville.append('Saint-Ouen'), dpt.append('Seine-Saint-Denis'), region.append('Île-de-France'), pays.append('France')
        elif v[0].title() == 'Brest': ville.append('Brest'), dpt.append('Finistère'), region.append('Bretagne'), pays.append('France')
        elif v[0].title() == 'Vitry-sur-Seine': ville.append('Vitry-sur-Seine'), dpt.append('Val-de-Marne'), region.append('Île-de-France'), pays.append('France')
        elif v[0].title() == 'Bangkok': ville.append('Bangkok'), dpt.append('Bangkok'), region.append('Bangkok'), pays.append('Thailande')
        elif v[0].title() == 'Henin Beaumont' or v[0].title() == 'Henin-Beaumont' or v[0].title() == 'Hénin-Beaumont': ville.append('Hénin-Beaumont'), dpt.append('Pas-de-Calais'), region.append('Hauts-de-France'), pays.append('France')
        elif v[0].title() == 'Amiens': ville.append('Amiens'), dpt.append('Somme'), region.append('Hauts-de-France'), pays.append('France')
        elif v[0].title() == 'Wissous':
            temp = geo.loc[geo['ville'] == f'Wissous{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Bayonne':
            temp = geo.loc[geo['ville'] == f'Bayonne{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Saint-Sébastien-Sur-Loire':
            temp = geo.loc[geo['ville'] == f'Saint-Sébastien-Sur-Loire{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Saint-Chamond':
            temp = geo.loc[geo['ville'] == f'Saint-Chamond{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Saint-Nazaire' or v[0].title() == 'Saint Nazaire':
            temp = geo.loc[geo['ville'] == f'Saint-Nazaire{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Lille':
            temp = geo.loc[geo['ville'] == f'Lille{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Marseille':
            temp = geo.loc[geo['ville'] == f'Marseille{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Velizy' or v[0].title() == 'Vélizy-Villacoublay':
            temp = geo.loc[geo['ville'] == f'Vélizy-Villacoublay{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Élancourt' or v[0].title() == 'Elancourt':
            temp = geo.loc[geo['ville'] == f'Élancourt{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Brea':
            temp = geo.loc[geo['ville'] == f'Brea{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Nantes':
            temp = geo.loc[geo['ville'] == f'Nantes{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Toulouse':
            temp = geo.loc[geo['ville'] == f'Toulouse{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Lorient':
            temp = geo.loc[geo['ville'] == f'Lorient{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Gennevilliers':
            temp = geo.loc[geo['ville'] == f'Gennevilliers{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Roanne':
            temp = geo.loc[geo['ville'] == f'Roanne{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Shanghai':
            temp = geo.loc[geo['ville'] == f'Shanghai{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Valenciennes':
            temp = geo.loc[geo['ville'] == f'Valenciennes{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Charleroi':
            temp = geo.loc[geo['ville'] == f'Charleroi{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Issy-Les-Moulineaux' or v[0].title() == 'Issy Les Moulineaux':
            temp = geo.loc[geo['ville'] == f'Issy-Les-Moulineaux{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Toronto':
            temp = geo.loc[geo['ville'] == f'Toronto{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Chorzow' or v[0].title() == 'Chorzów':
            temp = geo.loc[geo['ville'] == f'Chorzow{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Bucarest' or v[0].title() == 'Bucuresti' or v[0].title() == 'Bucaresti':
            temp = geo.loc[geo['ville'] == f'Bucarest{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Pau':
            temp = geo.loc[geo['ville'] == f'Pau{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Lyon':
            temp = geo.loc[geo['ville'] == f'Lyon{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Hengelo':
            temp = geo.loc[geo['ville'] == f'Hengelo{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Montreal' or v[0].title() == 'Montréal':
            temp = geo.loc[geo['ville'] == f'Montreal{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Cambridge':
            temp = geo.loc[geo['ville'] == f'Cambridge{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Noida':
            temp = geo.loc[geo['ville'] == f'Noida{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Ivry Sur Seine' or v[0].title() == 'Ivry':
            temp = geo.loc[geo['ville'] == f'Ivry Sur Seine{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Saint-Denis' or v[0].title() == 'Saint Denis':
            temp = geo.loc[geo['ville'] == f'Saint-Denis{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Toulon':
            temp = geo.loc[geo['ville'] == f'Toulon{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Bridgewater':
            temp = geo.loc[geo['ville'] == f'Bridgewater{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Annecy':
            temp = geo.loc[geo['ville'] == f'Annecy{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Arras':
            temp = geo.loc[geo['ville'] == f'Arras{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Nanterre':
            temp = geo.loc[geo['ville'] == f'Nanterre{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Madrid':
            temp = geo.loc[geo['ville'] == f'Madrid{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Berlin':
            temp = geo.loc[geo['ville'] == f'Berlin{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Singapore' or v[0].title() == 'Singapour':
            temp = geo.loc[geo['ville'] == f'Singapour{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Massy':
            temp = geo.loc[geo['ville'] == f'Massy{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Cholet':
            temp = geo.loc[geo['ville'] == f'Cholet{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Framingham':
            temp = geo.loc[geo['ville'] == f'Framingham{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Ollioules':
            temp = geo.loc[geo['ville'] == f'Ollioules{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Portet Sur Garonne':
            temp = geo.loc[geo['ville'] == f'Portet Sur Garonne{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Orléans':
            temp = geo.loc[geo['ville'] == f'Orléans{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'México' or v[0].title() == 'Mexico':
            temp = geo.loc[geo['ville'] == f'Mexico{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Rydalmere':
            temp = geo.loc[geo['ville'] == f'Rydalmere{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Melbourne':
            temp = geo.loc[geo['ville'] == f'Melbourne{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Grenoble':
            temp = geo.loc[geo['ville'] == f'Grenoble{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Sydney':
            temp = geo.loc[geo['ville'] == f'Sydney{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Savigliano':
            temp = geo.loc[geo['ville'] == f'Savigliano{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Karlstein':
            temp = geo.loc[geo['ville'] == f'Karlstein{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Stuttgart':
            temp = geo.loc[geo['ville'] == f'Stuttgart{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Rennes':
            temp = geo.loc[geo['ville'] == f'Rennes{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Le Havre':
            temp = geo.loc[geo['ville'] == f'Le Havre{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Montpellier':
            temp = geo.loc[geo['ville'] == f'Montpellier{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Auxerre':
            temp = geo.loc[geo['ville'] == f'Auxerre{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Niort':
            temp = geo.loc[geo['ville'] == f'Niort{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Tulle':
            temp = geo.loc[geo['ville'] == f'Tulle{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Colombelles':
            temp = geo.loc[geo['ville'] == f'Colombelles{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'La Chapelle Saint Ursin' or v[0].title() == 'La-Chapelle-Saint-Ursin':
            temp = geo.loc[geo['ville'] == f'La Chapelle Saint Ursin{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Angers':
            temp = geo.loc[geo['ville'] == f'Angers{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Rungis':
            temp = geo.loc[geo['ville'] == f'Rungis{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Caen':
            temp = geo.loc[geo['ville'] == f'Caen{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Rouen':
            temp = geo.loc[geo['ville'] == f'Rouen{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Nur-Sultan':
            temp = geo.loc[geo['ville'] == f'Nur-Sultan{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Prague':
            temp = geo.loc[geo['ville'] == f'Prague{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Temara':
            temp = geo.loc[geo['ville'] == f'Temara{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Oslo':
            temp = geo.loc[geo['ville'] == f'Oslo{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Swiftwater':
            temp = geo.loc[geo['ville'] == f'Swiftwater{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Atlanta':
            temp = geo.loc[geo['ville'] == f'Atlanta{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Laval':
            temp = geo.loc[geo['ville'] == f'Laval{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Barcelona' or v[0].title() == 'Barcelone':
            temp = geo.loc[geo['ville'] == f'Barcelone{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Soliman':
            temp = geo.loc[geo['ville'] == f'Soliman{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Limours':
            temp = geo.loc[geo['ville'] == f'Limours{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'New York':
            temp = geo.loc[geo['ville'] == f'New York{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Lisbonne':
            temp = geo.loc[geo['ville'] == f'Lisbonne{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Torino' or v[0].title() == 'Turin':
            temp = geo.loc[geo['ville'] == f'Torino{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Moscow' or v[0].title() == 'Moscou':
            temp = geo.loc[geo['ville'] == f'Moscou{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Francfort-Sur-Le-Main' or v[0].title() == 'Francfort Sur Le Main':
            temp = geo.loc[geo['ville'] == f'Francfort-Sur-Le-Main{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Lisieux':
            temp = geo.loc[geo['ville'] == f'Lisieux{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Eragny':
            temp = geo.loc[geo['ville'] == f'Eragny{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Blackburn':
            temp = geo.loc[geo['ville'] == f'Blackburn{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Courbevoie':
            temp = geo.loc[geo['ville'] == f'Courbevoie{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Ocoyoacac':
            temp = geo.loc[geo['ville'] == f'Ocoyoacac{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Copenhagen' or v[0].title() == 'Copenhague':
            temp = geo.loc[geo['ville'] == f'Copenhagen{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Puilboreau':
            temp = geo.loc[geo['ville'] == f'Puilboreau{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Zeralda':
            temp = geo.loc[geo['ville'] == f'Zeralda{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Saint Brieuc' or v[0].title() == 'Saint-Brieuc' or v[0].title() == 'St Brieuc':
            temp = geo.loc[geo['ville'] == f'Saint Brieuc{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Bagnolet':
            temp = geo.loc[geo['ville'] == f'Bagnolet{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Bagneux':
            temp = geo.loc[geo['ville'] == f'Bagneux{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Budapest':
            temp = geo.loc[geo['ville'] == f'Budapest{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Bordes':
            temp = geo.loc[geo['ville'] == f'Bordes{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Alençon':
            temp = geo.loc[geo['ville'] == f'Alençon{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Cleveland':
            temp = geo.loc[geo['ville'] == f'Cleveland{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Metz':
            temp = geo.loc[geo['ville'] == f'Metz{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Mantes La Jolie' or v[0].title() == 'Mantes-La-Jolie':
            temp = geo.loc[geo['ville'] == f'Mantes La Jolie{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Dampierre-En-Burly' or v[0].title() == 'Dampierre En Burly':
            temp = geo.loc[geo['ville'] == f'Dampierre-En-Burly{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'La Défense' or v[0].title() == 'La Defense':
            temp = geo.loc[geo['ville'] == f'La Défense{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Epinal' or v[0].title() == 'Épinal':
            temp = geo.loc[geo['ville'] == f'Epinal{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Aubervilliers':
            temp = geo.loc[geo['ville'] == f'Aubervilliers{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Détroit' or v[0].title() == 'Detroit':
            temp = geo.loc[geo['ville'] == f'Détroit{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Saran':
            temp = geo.loc[geo['ville'] == f'Saran{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Thionville':
            temp = geo.loc[geo['ville'] == f'Thionville{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Beauvais':
            temp = geo.loc[geo['ville'] == f'Beauvais{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Albertville':
            temp = geo.loc[geo['ville'] == f'Albertville{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Bourg En Bresse'or v[0].title() == 'Bourg-En-Bresse':
            temp = geo.loc[geo['ville'] == f'Bourg En Bresse{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Mérignac':
            temp = geo.loc[geo['ville'] == f'Mérignac{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Buenos Aires':
            temp = geo.loc[geo['ville'] == f'Buenos Aires{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Granville':
            temp = geo.loc[geo['ville'] == f'Granville{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Feyzin':
            temp = geo.loc[geo['ville'] == f'Feyzin{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Beaucouzé':
            temp = geo.loc[geo['ville'] == f'Beaucouzé{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Menton':
            temp = geo.loc[geo['ville'] == f'Menton{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Palaiseau':
            temp = geo.loc[geo['ville'] == f'Palaiseau{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Lannion':
            temp = geo.loc[geo['ville'] == f'Lannion{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Djeddah':
            temp = geo.loc[geo['ville'] == f'Djeddah{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Pierrelatte':
            temp = geo.loc[geo['ville'] == f'Pierrelatte{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Midrand':
            temp = geo.loc[geo['ville'] == f'Midrand{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Brive-La-Gaillarde' or v[0].title() == 'Brive La Gaillarde':
            temp = geo.loc[geo['ville'] == f'Brive-La-Gaillarde{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Coblence':
            temp = geo.loc[geo['ville'] == f'Coblence{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Langueux':
            temp = geo.loc[geo['ville'] == f'Langueux{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Nevers':
            temp = geo.loc[geo['ville'] == f'Nevers{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Johannesbourg':
            temp = geo.loc[geo['ville'] == f'Johannesbourg{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Rians':
            temp = geo.loc[geo['ville'] == f'Rians{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'St Sebastien':
            temp = geo.loc[geo['ville'] == f'St Sebastien{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Compiègne':
            temp = geo.loc[geo['ville'] == f'Compiègne{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Gignac':
            temp = geo.loc[geo['ville'] == f'Gignac{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Haguenau':
            temp = geo.loc[geo['ville'] == f'Haguenau{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Thonon-Les-Bains' or v[0].title() == 'Thonon Les Bains':
            temp = geo.loc[geo['ville'] == f'Thonon-Les-Bains{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Saumur':
            temp = geo.loc[geo['ville'] == f'Saumur{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Avignon':
            temp = geo.loc[geo['ville'] == f'Avignon{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'La Courneuve':
            temp = geo.loc[geo['ville'] == f'La Courneuve{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Eyguières':
            temp = geo.loc[geo['ville'] == f'Eyguières{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'St Maximin' or v[0].title() == 'Saint Maximin':
            temp = geo.loc[geo['ville'] == f'St Maximin{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Avranches':
            temp = geo.loc[geo['ville'] == f'Avranches{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Bouguenais':
            temp = geo.loc[geo['ville'] == f'Bouguenais{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Pantin':
            temp = geo.loc[geo['ville'] == f'Pantin{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Lens':
            temp = geo.loc[geo['ville'] == f'Lens{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Saint Dizier' or v[0].title() == 'Saint-Dizier':
            temp = geo.loc[geo['ville'] == f'Saint-Dizier{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Chauny':
            temp = geo.loc[geo['ville'] == f'Chauny{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Pont-Audemer' or v[0].title() == 'Pont Audemer':
            temp = geo.loc[geo['ville'] == f'Pont-Audemer{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Libourne':
            temp = geo.loc[geo['ville'] == f'Libourne{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Gardanne':
            temp = geo.loc[geo['ville'] == f'Gardanne{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Domerat':
            temp = geo.loc[geo['ville'] == f'Domerat{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Pôrto Alegre' or v[0].title() == 'Porto Alegre':
            temp = geo.loc[geo['ville'] == f'Pôrto Alegre{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Pontarlier':
            temp = geo.loc[geo['ville'] == f'Pontarlier{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Wattrelos': ville.append('Wattrelos'), dpt.append('Nord'), region.append('Hauts-de-France'), pays.append('France')
        elif v[0].title() == 'Charleville-Mézières' or v[0].title() == 'Charleville-Mezieres' or v[0].title() == 'Charleville Mézières' or v[0].title() == 'Charleville Mezieres':
            temp = geo.loc[geo['ville'] == f'Charleville-Mézières{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Charlotte':
            temp = geo.loc[geo['ville'] == f'Charlotte{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Abbeville':
            temp = geo.loc[geo['ville'] == f'Abbeville{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'München' or v[0].title() == 'Munich':
            temp = geo.loc[geo['ville'] == f'München{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Saint-Lô' or v[0].title() == 'Saint-Lo' or v[0].title() == 'Saint Lô' or v[0].title() == 'Saint Lo':
            temp = geo.loc[geo['ville'] == f'Saint Lô{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Carquefou':
            temp = geo.loc[geo['ville'] == f'Carquefou{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Austin':
            temp = geo.loc[geo['ville'] == f'Austin{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Dhari':
            temp = geo.loc[geo['ville'] == f'Soliman{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Canton':
            temp = geo.loc[geo['ville'] == f'Canton{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Cwmbran':
            temp = geo.loc[geo['ville'] == f'Cwmbran{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'La Rochelle':
            temp = geo.loc[geo['ville'] == f'La Rochelle{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'London':
            temp = geo.loc[geo['ville'] == f'London{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Reims':
            temp = geo.loc[geo['ville'] == f'Reims{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Magny-Les-Hameaux':
            temp = geo.loc[geo['ville'] == f'Magny-Les-Hameaux{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'La Queue-En-Brie':
            temp = geo.loc[geo['ville'] == f'La Queue-En-Brie{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Villeurbanne':
            temp = geo.loc[geo['ville'] == f'Villeurbanne{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Chevilly-Larue':
            temp = geo.loc[geo['ville'] == f'Chevilly-Larue{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Shenzhen':
            temp = geo.loc[geo['ville'] == f'Shenzhen{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Allston':
            temp = geo.loc[geo['ville'] == f'Allston{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Dieppe':
            temp = geo.loc[geo['ville'] == f'Dieppe{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Fécamp' or v[0].title() == 'Fecamp':
            temp = geo.loc[geo['ville'] == f'Fécamp{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Bologna':
            temp = geo.loc[geo['ville'] == f'Bologna{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Waltham':
            temp = geo.loc[geo['ville'] == f'Waltham{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Suzano':
            temp = geo.loc[geo['ville'] == f'Suzano{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Chengdu':
            temp = geo.loc[geo['ville'] == f'Chengdu{id_algo}']
            ville.append(temp['ville'].values[0]), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
        elif v[0].title() == 'Huntington Beach': ville.append('Huntington Beach'), dpt.append("Comté d'Orange"), region.append('Californie'), pays.append('États-Unis')
        else:
            t = re.findall('drome|drôme|gironde|gard|manche|eure|hauts-de-seine|seine et marne|savoie|haute savoie|isère|rhône|loire atlantique|saône et loire|bouches du rhône|hérault', target.lower())
            #england|angleterre|
            if not t:
                ville.append('')
                dpt.append('')
                region.append('TBD1')
                pays.append('TBD1')
            elif t[0].title() == 'Gironde': ville.append(''), dpt.append('Gironde'), region.append('Nouvelle-Aquitaine'), pays.append('France')
            elif t[0].title() == 'Drome' or t[0].title() == 'Drôme':
                temp = geo.loc[geo['ville'] == f'Drôme{id_algo}']
                ville.append(''), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
            elif t[0].title() == 'Gard':
                temp = geo.loc[geo['ville'] == f'Gard{id_algo}']
                ville.append(''), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
            elif t[0].title() == 'Manche':
                temp = geo.loc[geo['ville'] == f'Manche{id_algo}']
                ville.append(''), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
            elif t[0].title() == 'Eure':
                temp = geo.loc[geo['ville'] == f'Eure{id_algo}']
                ville.append(''), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
            elif t[0].title() == 'Hauts-De-Seine' or t[0].title() == 'Hauts De Seine' or t[0].title() == 'Hauts-de-Seine' or t[0].title() == 'Hauts de Seine':
                temp = geo.loc[geo['ville'] == f'Hauts-de-Seine{id_algo}']
                ville.append(''), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
            elif t[0].title() == 'Seine-Et-Marne' or t[0].title() == 'Seine Et Marne' or t[0].title() == 'Seine-et-Marne' or t[0].title() == 'Seine et Marne':
                temp = geo.loc[geo['ville'] == f'Seine-et-Marne{id_algo}']
                ville.append(''), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
            elif t[0].title() == 'Savoie':
                temp = geo.loc[geo['ville'] == f'Savoie{id_algo}']
                ville.append(''), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
            elif t[0].title() == 'Haute Savoie' or t[0].title() == 'Haute-Savoie':
                temp = geo.loc[geo['ville'] == f'Haute-Savoie{id_algo}']
                ville.append(''), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
            elif t[0].title() == 'Isère' or t[0].title() == 'Isere':
                temp = geo.loc[geo['ville'] == f'Isère{id_algo}']
                ville.append(''), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
            elif t[0].title() == 'Rhone' or t[0].title() == 'Rhône':
                temp = geo.loc[geo['ville'] == f'Rhône{id_algo}']
                ville.append(''), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
            elif t[0].title() == 'Loire-Atlantique' or t[0].title() == 'Loire Atlantique': ville.append(''), dpt.append('Loire Atlantique'), region.append('Pays de la Loire'), pays.append('France')
            elif t[0].title() == 'Saône et Loire':
                temp = geo.loc[geo['ville'] == f'Saône et Loire{id_algo}']
                ville.append(''), dpt.append(temp['dpt'].values[0]), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
            elif t[0].title() == 'Bouches du Rhône': ville.append(''), dpt.append('Bouches du Rhône'), region.append('Provence-Alpes-Côte d’Azur'), pays.append('France')
            else:
                r = re.findall("géorgie|bade-wurtemberg|michigan|overijssel|new york|caroline du nord|lancashire|nouvelle-galles du sud|thessaloniki|rhénanie-palatinat|californie|ontario|rio grande do sul|new jersey|uttar pradesh|massachusetts|wallonie|île-de-france|normandie|bretagne|ile de france|hauts de france|centre val de loire|provence/alps|hauts-de-france|quebec|québec|auvergne-rhône-alpes|bourgogne franche comte|pays de la loire|provence-alpes-côte d'azur|grand est|nouvelle-aquitaine|occitanie", target.lower())
                if not r:
                    ville.append('')
                    dpt.append('')
                    region.append('')
                    pays.append('TBD2')
                elif r[0].title() == 'Michigan':
                    temp = geo.loc[geo['region'] == f'Michigan{id_algo}']
                    ville.append(''), dpt.append(''), region.append(temp['region'].values[0]), pays.append(temp['pays'].values[0])
                elif r[0].title() == 'New York': ville.append(''), dpt.append(''), region.append('New York'), pays.append('États-Unis')
                else:
                    c = re.findall('pologne|argentine|thaïlande|afrique du sud|sweden|ecosse|germany|australia|kazakhstan|irlande|hongrie|arabie saoudite|mexico|singapour|spain|algérie|mexique|england|angleterre|espagne|greece|belgium|united kingdom|allemagne|royaume-uni|norway|deutschland|egypt|singapore|italy|denmark|russie|morocco|brazil|france|canada|tunisia|chine|china|india|romania|états-unis|united states|poland', target.lower())
                    #|irlande|hongrie|arabie saoudite||||algérie||
                    #|greece||||norway
                    #||egypt|||denmark|russie|morocco|brazil|||
                    #|||romania
                    
                    if not r:
                        pays.append('TBD3')
                    elif r[0].title() == 'France': ville.append(''), dpt.append(''), region.append(''), pays.append('France')
                    elif r[0].title() == 'Pologne' or r[0].title() == 'Poland': ville.append(''), dpt.append(''), region.append(''), pays.append('Pologne')
                    elif r[0].title() == 'Argentine' or r[0].title() == 'Argentina': ville.append(''), dpt.append(''), region.append(''), pays.append('Argentine')
                    elif r[0].title() == 'Thaïlande' or r[0].title() == 'Thailand': ville.append(''), dpt.append(''), region.append(''), pays.append('Thaïlande')
                    elif r[0].title() == 'Afrique du sud': ville.append(''), dpt.append(''), region.append(''), pays.append('Afrique du Sud')
                    elif r[0].title() == 'Sweden' or r[0].title() == 'Suède': ville.append(''), dpt.append(''), region.append(''), pays.append('Suède')
                    elif r[0].title() == 'Ecosse' or r[0].title() == 'Scotland': ville.append(''), dpt.append(''), region.append(''), pays.append('Écosse')
                    elif r[0].title() == 'Germany' or r[0].title() == 'Allemagne' or r[0].title() == 'Deutschland': ville.append(''), dpt.append(''), region.append(''), pays.append('Allemagne')
                    elif r[0].title() == 'Australia' or r[0].title() == 'Australie': ville.append(''), dpt.append(''), region.append(''), pays.append('Australie')
                    elif r[0].title() == 'États-Unis' or r[0].title() == 'United States': ville.append(''), dpt.append(''), region.append(''), pays.append('États-Unis')
                    elif r[0].title() == 'United Kingdom' or r[0].title() == 'Royaume-Uni': ville.append(''), dpt.append(''), region.append(''), pays.append('Royaume-Uni')
                    elif r[0].title() == 'Mexico' or r[0].title() == 'Mexique': ville.append(''), dpt.append(''), region.append(''), pays.append('Mexique')
                    elif r[0].title() == 'Kazakhstan': ville.append(''), dpt.append(''), region.append(''), pays.append('Kazakhstan')
                    elif r[0].title() == 'Chine' or r[0].title() == 'China': ville.append(''), dpt.append(''), region.append(''), pays.append('Chine')
                    elif r[0].title() == 'Espagne' or r[0].title() == 'Spain': ville.append(''), dpt.append(''), region.append(''), pays.append('Espagne')
                    elif r[0].title() == 'Italy' or r[0].title() == 'Italie': ville.append(''), dpt.append(''), region.append(''), pays.append('Italie')
                    elif r[0].title() == 'Singapore' or r[0].title() == 'Singapour': ville.append(''), dpt.append(''), region.append(''), pays.append('Singapour')
                    elif r[0].title() == 'Canada': ville.append(''), dpt.append(''), region.append(''), pays.append('Canada')
                    elif r[0].title() == 'India' or r[0].title() == 'Inde': ville.append(''), dpt.append(''), region.append(''), pays.append('Inde')
                    elif r[0].title() == 'Belgique' or r[0].title() == 'Belgium': ville.append(''), dpt.append(''), region.append(''), pays.append('Belgique')
                    elif r[0].title() == 'Tunisie' or r[0].title() == 'Tunisia': ville.append(''), dpt.append(''), region.append(''), pays.append('Tunisie')
                    else:
                        dpt.append('No Info'), region.append('No Info'), pays.append('No Info')
    x = x + 1

df['country'] = pays
df['region'] = region
df['departement'] = dpt
df['ville'] = ville

In [10]:
df['country'].value_counts()

                      2067
France                 915
France [x]             890
Inde                   147
Espagne [x]             52
États-Unis [x]          44
United-Kingdom [x]      31
Roumanie [x]            28
Égypte                  23
Italie [x]              22
Mexique [x]             15
Chine [x]               14
Shanghai [x]            14
Canada [x]              14
Allemagne [x]           13
États-Unis              13
Pologne [x]             13
Thailande               11
Australie [x]            9
TBD1                     6
Hongrie [x]              6
Singapour [x]            6
Russie [x]               6
Danemark [x]             4
Tchéquie [x]             3
Brésil [x]               3
Norvège [x]              2
Argentine [x]            1
Kazakhstan [x]           1
Tunisie [x]              1
Algerie [x]              1
Afrique du Sud [x]       1
Name: country, dtype: int64

In [12]:
df['region'].value_counts()

                        2067
Île-de-France            810
Île-de-France [x]        208
Occitanie [x]            175
Karnataka                147
                        ... 
Gauteng [x]                1
District Féderal [x]       1
Nabeul [x]                 1
Oblys d'Aqmola [x]         1
Alger [x]                  1
Name: region, Length: 61, dtype: int64

In [14]:
df['ville'].value_counts()

                       2073
Paris                   331
Le Plessis-Robinson     323
Toulouse [x]            174
Bangalore               147
                       ... 
Pantin [x]                1
Eragny [x]                1
Charlotte [x]             1
Buenos Aires [x]          1
Auxerre [x]               1
Name: ville, Length: 129, dtype: int64

In [ ]:
# //////
# Job Field Allocation

In [10]:
df['details'][0]

'Toulouse Area, France'

In [11]:
df['function'][0]

'Qualification & Operability'

In [ ]:
df['details'].value_counts()

In [15]:
df['function'].value_counts().head(10)

Engineering                         380
Industrial                           76
Others                               70
Project & Bid Management             48
IS&T                                 44
Quality & Continuous Improvement     34
Supply Chain & Planning              27
Marketing                            25
Manufacturing                        24
Etudes et ingénierie                 24
Name: function, dtype: int64

In [16]:
df['function'] = df['function'].astype(str)
df['details'] = df['details'].astype(str)

In [17]:
job_field_type = []
x = 0
while x <= df.shape[0]-1:
    target = df['function'][x]
    
    c = re.findall("industrial|pilotage industriel|services industriels|installation-construction|building & offices|security|customer account and service management|sales|marketing|commercial & marketing|strategy, marketing and sales|commerce, marketing , offre|informatique produit|software engineering|manufacturing engineering|méthode et industrialisation|manufacturing|direction de l'entreprise|legal|ethics & compliance|leadership|enterprise risk & opportunity management|corporate social responsibility|Testing|elec.electron.&electromag,optics&optron.|electronique|electronical manufacturing|electricité|electrician|environment, heath & safety|sst-env|health safety & environment|global business support|client services|customer eng.&technical support&services|qualification & operability|nucléaire|pyrotechnie|sub-system development|creation & design|computing&comm and info& data processing|vehicle mission & control|communication|aménagement, peinture et composites|others|technical data & publications|flight line|training support & services|structure design & integration|quality inspection|materials & processes|flight & space physics|system installation|chaudronnerie|coque|production means/maintenance|soutien production|production|ingénierie/support production|chief engineering & architecture|programmes|etudes et ingénierie|programme & project management|project & bid management|engineering|sourcing, buying and ordering|is&t|logistics|information mgmt and digital technology|it|système d'information d'entreprise|supply chain & planning|supply chain & logistics|supply chain|logistique, supply chain|sourcing|achats|achat|human resources|ressources humaines|hr expertise|qualité|qualité & progrès|quality & continuous improvement|quality management system|finance|finances|accounting", target.lower())
    if not c:
        job_field_type.append(c)
        #print(c)
    else:
        if c[0].upper() == 'IS&T' or c[0].upper() == 'INFORMATION MGMT AND DIGITAL TECHNOLOGY' or c[0].upper() == 'IT' or c[0].lower() == "système d'information d'entreprise":
            job_field_type.append('IT')
        elif c[0].lower() == 'sourcing, buying and ordering' or c[0].upper() == 'SUPPLY CHAIN & PLANNING' or c[0].upper() == 'SUPPLY CHAIN & LOGISTICS' or c[0].upper() == 'SUPPLY CHAIN' or c[0].lower() == 'logistics' or c[0].upper() == 'LOGISTIQUE, SUPPLY CHAIN':
            job_field_type.append('Logistics & Supply Chain')
        elif c[0].upper() == 'SOURCING' or c[0].upper() == 'ACHATS' or c[0].upper() == 'ACHAT':
            job_field_type.append('Achat, Sourcing')
        elif c[0].upper() == 'HUMAN RESOURCES' or c[0].upper() == 'RESSOURCES HUMAINES' or c[0].upper() == 'HR EXPERTISE':
            job_field_type.append('Human Resources')
        elif c[0].lower() == 'qualité' or c[0].lower() == 'qualité & progrès' or c[0].lower() == 'quality & continuous improvement' or c[0].lower() == 'quality management system':
            job_field_type.append('Quality')
        elif c[0].lower() == 'finance' or c[0].lower() == 'finances' or c[0].lower() == 'accounting':
            job_field_type.append('Finance')
        elif c[0].lower() == 'engineering' or c[0].lower() == 'etudes et ingénierie' or c[0].lower() == 'chief engineering & architecture':
            job_field_type.append('Engineering')
        elif c[0].lower() == 'programmes' or c[0].lower() == 'programme & project management' or c[0].lower() == 'project & bid management':
            job_field_type.append('Programme and Project Management')
        elif c[0].lower() == 'production means/maintenance' or c[0].lower() == 'production' or c[0].lower() == 'soutien production' or c[0].lower() == 'ingénierie/support production':
            job_field_type.append('Production')
        elif c[0].lower() == 'industrial' or c[0].lower() == 'pilotage industriel' or c[0].lower() == 'services industriels':
            job_field_type.append('Industrial')
        elif c[0].lower() == 'installation-construction' or c[0].lower() == 'building & offices':
            job_field_type.append('Facility Management')
        elif c[0].lower() == 'security':
            job_field_type.append('Security')
        elif c[0].lower() == 'customer account and service management' or c[0].lower() == 'sales' or c[0].lower() == 'marketing' or c[0].lower() == 'commercial & marketing' or c[0].lower() == 'strategy, marketing and sales' or c[0].lower() == 'commerce, marketing , offre':
            job_field_type.append('Sales and Marketing')
        elif c[0].lower() == 'informatique produit' or c[0].lower() == 'software engineering':
            job_field_type.append('Software Engineering')
        elif c[0].lower() == 'manufacturing engineering' or c[0].lower() == 'méthode et industrialisation' or c[0].lower() == 'manufacturing':
            job_field_type.append('Manufacturing Engineering')
        elif c[0].lower() == "direction de l'entreprise" or c[0].lower() == 'legal' or c[0].lower() == 'ethics & compliance' or c[0].lower() == 'leadership' or c[0].lower() == 'enterprise risk & opportunity management' or c[0].lower() == 'corporate social responsibility':
            job_field_type.append('Legal & Company Management')
        elif c[0].lower() == 'soutien général':
            job_field_type.append('Customer Logistics Support')
        elif c[0].lower() == 'mécanique':
            job_field_type.append('Mechanical Engineering')
        elif c[0].lower() == 'contrôle, mesure, essais' or c[0].lower() == 'testing':
            job_field_type.append('Testing')
        elif c[0].lower() == 'elec.electron.&electromag,optics&optron.' or c[0].lower() == 'electronique' or c[0].lower() == 'electronical manufacturing' or c[0].lower() == 'electricité' or c[0].lower() == 'electrician':
            job_field_type.append('Electronical Engineering')
        elif c[0].lower() == 'environment, heath & safety' or c[0].lower() == 'sst-env' or c[0].lower() == 'health safety & environment':
            job_field_type.append('HSE')
        elif c[0].lower() == 'global business support' or c[0].lower() == 'client services' or c[0].lower() == 'customer eng.&technical support&services':
            job_field_type.append('Customer Services and Support')
        elif c[0].lower() == 'qualification & operability':
            job_field_type.append('Qualification & Operability')
        elif c[0].lower() == 'nucléaire' or c[0].lower() == 'pyrotechnie':
            job_field_type.append('Nuclear and Pyrotechnics')
        elif c[0].lower() == 'sub-system development':
            job_field_type.append('Sub-system development')
        elif c[0].lower() == 'creation & design':
            job_field_type.append('Creation & Design')
        elif c[0].lower() == 'computing&comm and info& data processing':
            job_field_type.append('Computing&Comm and Info& Data Processing')
        elif c[0].lower() == 'vehicle mission & control':
            job_field_type.append('Vehicle Mission & Control')
        elif c[0].lower() == 'communication':
            job_field_type.append('Communication')
        elif c[0].lower() == 'aménagement, peinture et composites':
            job_field_type.append('Aménagement, peinture et composites')
        elif c[0].lower() == 'others':
            job_field_type.append('Others')
        elif c[0].lower() == 'technical data & publications':
            job_field_type.append('Technical Data & Publications')
        elif c[0].lower() == 'flight line':
            job_field_type.append('Flight line')
        elif c[0].lower() == 'training support & services':
            job_field_type.append('Training support & services')
        elif c[0].lower() == 'structure design & integration':
            job_field_type.append('Structure Design & Integration')
        elif c[0].lower() == 'quality inspection':
            job_field_type.append('Quality Inspection')
        elif c[0].lower() == 'materials & processes':
            job_field_type.append('Materials & Processes')
        elif c[0].lower() == 'flight & space physics':
            job_field_type.append('Flight & Space Physics')
        elif c[0].lower() == 'system installation':
            job_field_type.append('System Installation')
        elif c[0].lower() == 'chaudronnerie':
            job_field_type.append('Chaudronnerie')
        elif c[0].lower() == 'coque':
            job_field_type.append('Coque')
        else:
            job_field_type.append(c[0].upper())
            
        
    x = x + 1

df['JOB_FIELD_1'] = job_field_type

job_field_type = []
x = 0
while x <= df.shape[0]-1:
    target = df['details'][x]
    
    c = re.findall("industrial|pilotage industriel|services industriels|installation-construction|building & offices|security|customer account and service management|sales|marketing|commercial & marketing|strategy, marketing and sales|commerce, marketing , offre|informatique produit|software engineering|manufacturing engineering|méthode et industrialisation|manufacturing|direction de l'entreprise|legal|ethics & compliance|leadership|enterprise risk & opportunity management|corporate social responsibility|Testing|elec.electron.&electromag,optics&optron.|electronique|electronical manufacturing|electricité|electrician|environment, heath & safety|sst-env|health safety & environment|global business support|client services|customer eng.&technical support&services|qualification & operability|nucléaire|pyrotechnie|sub-system development|creation & design|computing&comm and info& data processing|vehicle mission & control|communication|aménagement, peinture et composites|others|technical data & publications|flight line|training support & services|structure design & integration|quality inspection|materials & processes|flight & space physics|system installation|chaudronnerie|coque|production means/maintenance|soutien production|production|ingénierie/support production|chief engineering & architecture|programmes|etudes et ingénierie|programme & project management|project & bid management|engineering|sourcing, buying and ordering|is&t|logistics|information mgmt and digital technology|it|système d'information d'entreprise|supply chain & planning|supply chain & logistics|supply chain|logistique, supply chain|sourcing|achats|achat|human resources|ressources humaines|hr expertise|qualité|qualité & progrès|quality & continuous improvement|quality management system|finance|finances|accounting", target.lower())
    if not c:
        job_field_type.append(c)
        #print(c)
    else:
        if c[0].upper() == 'IS&T' or c[0].upper() == 'INFORMATION MGMT AND DIGITAL TECHNOLOGY' or c[0].upper() == 'IT' or c[0].lower() == "système d'information d'entreprise":
            job_field_type.append('IT')
        elif c[0].lower() == 'sourcing, buying and ordering' or c[0].upper() == 'SUPPLY CHAIN & PLANNING' or c[0].upper() == 'SUPPLY CHAIN & LOGISTICS' or c[0].upper() == 'SUPPLY CHAIN' or c[0].lower() == 'logistics' or c[0].upper() == 'LOGISTIQUE, SUPPLY CHAIN':
            job_field_type.append('Logistics & Supply Chain')
        elif c[0].upper() == 'SOURCING' or c[0].upper() == 'ACHATS' or c[0].upper() == 'ACHAT':
            job_field_type.append('Achat, Sourcing')
        elif c[0].upper() == 'HUMAN RESOURCES' or c[0].upper() == 'RESSOURCES HUMAINES' or c[0].upper() == 'HR EXPERTISE':
            job_field_type.append('Human Resources')
        elif c[0].lower() == 'qualité' or c[0].lower() == 'qualité & progrès' or c[0].lower() == 'quality & continuous improvement' or c[0].lower() == 'quality management system':
            job_field_type.append('Quality')
        elif c[0].lower() == 'finance' or c[0].lower() == 'finances' or c[0].lower() == 'accounting':
            job_field_type.append('Finance')
        elif c[0].lower() == 'engineering' or c[0].lower() == 'etudes et ingénierie' or c[0].lower() == 'chief engineering & architecture':
            job_field_type.append('Engineering')
        elif c[0].lower() == 'programmes' or c[0].lower() == 'programme & project management' or c[0].lower() == 'project & bid management':
            job_field_type.append('Programme and Project Management')
        elif c[0].lower() == 'production means/maintenance' or c[0].lower() == 'production' or c[0].lower() == 'soutien production' or c[0].lower() == 'ingénierie/support production':
            job_field_type.append('Production')
        elif c[0].lower() == 'industrial' or c[0].lower() == 'pilotage industriel' or c[0].lower() == 'services industriels':
            job_field_type.append('Industrial')
        elif c[0].lower() == 'installation-construction' or c[0].lower() == 'building & offices':
            job_field_type.append('Facility Management')
        elif c[0].lower() == 'security':
            job_field_type.append('Security')
        elif c[0].lower() == 'customer account and service management' or c[0].lower() == 'sales' or c[0].lower() == 'marketing' or c[0].lower() == 'commercial & marketing' or c[0].lower() == 'strategy, marketing and sales' or c[0].lower() == 'commerce, marketing , offre':
            job_field_type.append('Sales and Marketing')
        elif c[0].lower() == 'informatique produit' or c[0].lower() == 'software engineering':
            job_field_type.append('Software Engineering')
        elif c[0].lower() == 'manufacturing engineering' or c[0].lower() == 'méthode et industrialisation' or c[0].lower() == 'manufacturing':
            job_field_type.append('Manufacturing Engineering')
        elif c[0].lower() == "direction de l'entreprise" or c[0].lower() == 'legal' or c[0].lower() == 'ethics & compliance' or c[0].lower() == 'leadership' or c[0].lower() == 'enterprise risk & opportunity management' or c[0].lower() == 'corporate social responsibility':
            job_field_type.append('Legal & Company Management')
        elif c[0].lower() == 'soutien général':
            job_field_type.append('Customer Logistics Support')
        elif c[0].lower() == 'mécanique':
            job_field_type.append('Mechanical Engineering')
        elif c[0].lower() == 'contrôle, mesure, essais' or c[0].lower() == 'testing':
            job_field_type.append('Testing')
        elif c[0].lower() == 'elec.electron.&electromag,optics&optron.' or c[0].lower() == 'electronique' or c[0].lower() == 'electronical manufacturing' or c[0].lower() == 'electricité' or c[0].lower() == 'electrician':
            job_field_type.append('Electronical Engineering')
        elif c[0].lower() == 'environment, heath & safety' or c[0].lower() == 'sst-env' or c[0].lower() == 'health safety & environment':
            job_field_type.append('HSE')
        elif c[0].lower() == 'global business support' or c[0].lower() == 'client services' or c[0].lower() == 'customer eng.&technical support&services':
            job_field_type.append('Customer Services and Support')
        elif c[0].lower() == 'qualification & operability':
            job_field_type.append('Qualification & Operability')
        elif c[0].lower() == 'nucléaire' or c[0].lower() == 'pyrotechnie':
            job_field_type.append('Nuclear and Pyrotechnics')
        elif c[0].lower() == 'sub-system development':
            job_field_type.append('Sub-system development')
        elif c[0].lower() == 'creation & design':
            job_field_type.append('Creation & Design')
        elif c[0].lower() == 'computing&comm and info& data processing':
            job_field_type.append('Computing&Comm and Info& Data Processing')
        elif c[0].lower() == 'vehicle mission & control':
            job_field_type.append('Vehicle Mission & Control')
        elif c[0].lower() == 'communication':
            job_field_type.append('Communication')
        elif c[0].lower() == 'aménagement, peinture et composites':
            job_field_type.append('Aménagement, peinture et composites')
        elif c[0].lower() == 'others':
            job_field_type.append('Others')
        elif c[0].lower() == 'technical data & publications':
            job_field_type.append('Technical Data & Publications')
        elif c[0].lower() == 'flight line':
            job_field_type.append('Flight line')
        elif c[0].lower() == 'training support & services':
            job_field_type.append('Training support & services')
        elif c[0].lower() == 'structure design & integration':
            job_field_type.append('Structure Design & Integration')
        elif c[0].lower() == 'quality inspection':
            job_field_type.append('Quality Inspection')
        elif c[0].lower() == 'materials & processes':
            job_field_type.append('Materials & Processes')
        elif c[0].lower() == 'flight & space physics':
            job_field_type.append('Flight & Space Physics')
        elif c[0].lower() == 'system installation':
            job_field_type.append('System Installation')
        elif c[0].lower() == 'chaudronnerie':
            job_field_type.append('Chaudronnerie')
        elif c[0].lower() == 'coque':
            job_field_type.append('Coque')
        else:
            job_field_type.append(c[0].upper())
            
        
    x = x + 1

df['JOB_FIELD_2'] = job_field_type



In [19]:
df['JOB_FIELD_2'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[]                                  3703
IT                                   437
Sales and Marketing                   49
Engineering                           25
Production                            23
Manufacturing Engineering             18
Software Engineering                  18
Industrial                            18
Human Resources                       16
Electronical Engineering              11
Programme and Project Management      10
Achat, Sourcing                       10
Quality                               10
Logistics & Supply Chain              10
Finance                                5
Communication                          4
Security                               2
Nuclear and Pyrotechnics               2
HSE                                    2
Coque                                  1
Legal & Company Management             1
Customer Services and Support          1
Name: JOB_FIELD_2, dtype: int64

In [17]:
df['JOB_FIELD_2'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[]                                  3368
IT                                   390
Sales and Marketing                   47
Engineering                           23
Production                            21
Manufacturing Engineering             18
Software Engineering                  16
Human Resources                       15
Industrial                            14
Achat, Sourcing                       10
Programme and Project Management       9
Quality                                8
Logistics & Supply Chain               7
Electronical Engineering               6
Finance                                5
Communication                          3
HSE                                    2
Security                               2
Coque                                  1
Customer Services and Support          1
Legal & Company Management             1
Name: JOB_FIELD_2, dtype: int64

In [ ]:
""



///////////////
******
///////////////

'Engineering',
 'Industrial',
 'IS&T',
 'Project & Bid Management',
 'Etudes et ingénierie',
 'Quality & Continuous Improvement',
 'Finance',
 'Supply Chain & Planning',
 'Security',
 'Installation-Construction',
 'Sourcing',
 'Manufacturing',
 'Sales',
 'Manufacturing Engineering',
 'Programmes',
 'Marketing',
 'Supply Chain & Logistics',
 'Information Mgmt and Digital Technology',
 'Informatique produit',
 'Human Resources',
 'Legal',
 'Elec.Electron.&Electromag,Optics&Optron.',
 'Others',
 'Méthode et Industrialisation',
 'Qualification & Operability',
 'Programme & Project Management',
 'Electricité',
 "Système d'information d'entreprise",
 'Chief Engineering & Architecture',
 'Contrôle, mesure, essais',
 'Software Engineering',
 'Flight line',
 'Ethics & Compliance',
 'Services industriels',
 'Pilotage Industriel',
 'Mécanique',
 'Soutien général',
 'Environment, Heath & Safety',
 'Supply Chain',
 'Production',
 'Vehicle Mission & Control',
 'Electronique',
 'Strategy, Marketing and Sales',
 'Qualité',
 'Qualité & progrès',
 'Building & Offices',
 'Client Services',
 'Nucléaire',
 'Computing&Comm and Info& Data Processing',
 'Quality Inspection',
 'Ressources humaines',
 'Commerce, marketing , Offre',
 'Electronical manufacturing',
 'Engie Solutions',
 'IT',
 'Testing',
 'Sub-system development',
 'Leadership',
 'Commercial & Marketing',
 'Chaudronnerie',
 'Creation & Design',
 'Communication',
 'Logistique, Supply chain',
 'Customer Eng.&Technical Support&Services',
 'Achats',
 'HR Expertise',
 'Assembly & Integration',
 'Engie Impact',
 'Generation Europe',
 'Materials & Processes',
 'Sourcing, Buying and Ordering',
 'Pyrotechnie',
 'System Installation',
 'LEAN/Amélioration continue',
 'SST-ENV',
 'Structure & Dynamic Systems Analysis',
 "Direction de l'entreprise",
 'Sales, Marketing & Commercial Contracts',
 'Coque',
 'Finances',
 'Enterprise Risk & Opportunity Management',
 'Ingénierie/Support production',
 'Accounting',
 'Structure Design & Integration',
 'Ressources Humaines',
 'Technical Data & Publications',
 'Soutien production',
 'Test operations',
 'North America',
 'Achat',
 'France BtoB',
 'Powerplant and Propulsion',
 'Benelux',
 'Electrician',
 'Material Support & services',
 'Mechanics & Mechatronics',
 'Production means/Maintenance',
 'Customer Account and Service Management',
 'Flight & Space Physics',
 'Aménagement, peinture et composites',
 'Soudage',
 'France Energies Renouvelables',
 'Global Business Support',
 'Fluid & Hydromechanics, Thermal',
 'Management',
 'Configuration Management',
 'Logistics',
 'Company Communication',
 'Tractebel',
 'Training support & services',
 'GRDF',
 'Quality Management System',
 'Services généraux/Maintenance',
 'Security & Product Safety',
 'Corporate Social Responsibility',
 'Organisation',
 'Management de la donnée et systèmes d’information',
 'Production Planning & Scheduling',
 'HR Business Partnership',
 ]

In [ ]:
df.iloc[:50,-2:]

In [22]:
df['JOB_FIELD_1'].head()

0    []
1    []
2    []
3    []
4    []
Name: JOB_FIELD_1, dtype: object

In [ ]:
target = df['function'][0].lower()

In [ ]:
re.findall("is&t|information mgmt and digital technology|it|système d'information d'entreprise|supply chain & planning|supply chain & logistics|supply chain|logistique, supply chain|sourcing|achats|achat|human resources|ressources humaines|hr expertise|qualité|qualité & progrès|quality & continuous improvement|quality management system|finance|finances|accounting", target.lower())

In [23]:
df = df.drop_duplicates(subset=['link'])

In [20]:
# Check for doubles in Links
for x in df['link']:
    temp = df.loc[df['link'] == x, :]
    if len(temp) > 1:
        print(x, temp['name'])

In [ ]:
df.loc[1283]

In [ ]:
df.loc[3610]

In [ ]:
from collections import Counter
import re

wordlist = []
i = 0
while i < len(df_temp['details'].value_counts().index):
    for x in df_temp['details'].value_counts().index.values[i].split(' '):
        if x == '' or x == '/*/' or x == '/' or x == ':' or x == 'Ref.' or x == '-':
            pass
        elif re.compile(r'^(0[1-9]|[12][0-9]|3[01])[- /.](0[1-9]|1[012])[- /.](19|20)\d\d$').match(x):
            pass
        elif re.compile(r'^(0[1-9]|1[012])[- /.](0[1-9]|[12][0-9]|3[01])[- /.](19|20)\d\d$').match(x):
            pass
        elif re.compile(r'^(0[1-9]|[12][0-9]|3[01])[- /.]([1-9]|1[012])[- /.](19|20)\d\d$').match(x):
            pass
        elif re.compile(r'^([1-9]|1[012])[- /.](0[1-9]|[12][0-9]|3[01])[- /.](19|20)\d\d$').match(x):
            pass
        elif re.compile(r'^([1-9]|1[012])[- /.]([1-9]|[12][0-9]|3[01])[- /.](19|20)\d\d$').match(x):
            pass
        elif re.compile(r'\(.*\)$').match(x):
            pass
        else:
            wordlist.append(x)
    i = i + 1

value_count = Counter(wordlist)
value_count.most_common()

In [ ]:
wordlist_cleaned = []
txt = re.split('\t| |,|/|•|\.', df_temp['desc'][4])
wordlist = set(txt)

deletion_table = ['"', '&', '(', ')', '?',':','!', '*', ',', '/', ]

for word in wordlist:
    for char in deletion_table:
        word = word.replace(char, "")
    nb = len(re.findall(word, ' '.join(txt)))
    wordlist_cleaned.append([word, nb])

In [ ]:
wordlist_cleaned

In [24]:
df.to_csv('offers_v3_cleaned.csv')

In [23]:
df.head()

name  \
0     INGENIEUR ASSURANCE CONCEPTION ELECTRONIQUE (H/F)   
1    Assistant Ressources Humaines (H/F) - Orléans (45)   
2     Ingénieur électronique analogique/puissance (h/f)   
3  Ingénieur(e) électronique analogique/puissance (h/f)   
4                   Enterprise Security Architect (m/f)   

                                                                                                  link  \
0      https://www.airbus.com/careers/search-and-apply/search-for-vacancies/job-detail.html?uuid=12697   
1  https://joinus.saint-gobain.com/fr/fra/hr/p/11650/33519/assistant-ressources-humaines-hf-orleans-45   
2      https://www.airbus.com/careers/search-and-apply/search-for-vacancies/job-detail.html?uuid=18265   
3      https://www.airbus.com/careers/search-and-apply/search-for-vacancies/job-detail.html?uuid=18264   
4      https://www.airbus.com/careers/search-and-apply/search-for-vacancies/job-detail.html?uuid=18262   

        company                                  function  \
0        Airbus               Qualification & Operability   
1  Saint Gobain                                       nan   
2        Airbus  Elec.Electron.&Electromag,Optics&Optron.   
3        Airbus  Elec.Electron.&Electromag,Optics&Optron.   
4        Airbus                                  Security   

                                                                                                     details  \
0                                                                                      Toulouse Area, France   
1  FRA05570 /*/ Assistant Ressources Humaines (H/F) - Orléans (45) France Centre-Val de Loire CDI 17/06/2021   
2                                                                                         Paris Area, France   
3                                                                                         Paris Area, France   
4                                                                                      Toulouse Area, France   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   